In [1]:
import gym # type: ignore
import random
import numpy as np # type: ignore
from tensorflow.keras.layers import Dense , Flatten # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore 
from rl.agents import DQNAgent # type: ignore
from rl.policy import BoltzmannQPolicy # type: ignore
from rl.memory import SequentialMemory # type: ignore


c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py:546: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class IteratorBase(collections.Iterator, trackable.Trackable,
c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:106: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class DatasetV2(collections.Iterable, tracking_base.Trackable,
c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\tensorflow\python\autograph\utils\testing.py:21: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [24]:
# env = gym.make('CartPole-v1', render_mode='human') # if you want to render the environment and visualize it
env = gym.make('CartPole-v1') # if you don't want to render the environment for training purpose

states = env.observation_space.shape[0] 
actions = env.action_space.n 

c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [25]:
# 2 actions : 0 left or 1 right
actions

2

In [26]:
# 4 states  : 
# Cart position: The position of the cart on the track (continuous value).
# Cart velocity: The speed of the cart (continuous value).
# Pole angle: The angle of the pole relative to the vertical position (continuous value).
# Pole angular velocity: The rate of change of the pole's angle (continuous value).
states

4

In [27]:
# env = gym.make('CartPole-v1', render_mode='human') # if you want to render the environment and visualize it

# episodes = 10
# for episode in range(1, episodes+1):
#           state = env.reset() # reset  the environment
#           done = False 
#           score = 0
          
#           while not done:
#                     env.render() # render the environment
#                     action = random.choice([0,1]) # random action
#                     n_state, reward, done , info = env.step(action) # take action ( "_" is for truncate )
#                     score+=reward
#           print('Episode:{} Score:{}'.format(episode, score))
          
# env.close()

# # we notice that the cartpole is not stable and falls down quickly (the max score is 500)

In [28]:
def create_model(states, actions):
    model = Sequential([
              Flatten(input_shape=(1,states)), # it like Input but it make 3D to 2D (to convert imgs to vectors)
              Dense(24, activation='relu'),
              Dense(24, activation='relu'),
              Dense(actions, activation='linear') # softmax is not used in this case because we want to maximize the reward (0 or 1) if we wanna make it a softmax we will have  to change the  activation function to softmax or make the  other method  that i learn (it's better) 
    ]  , name='CartPoleModel')
    return model

In [29]:
model = create_model(states, actions)

model.summary()

Model: "CartPoleModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [30]:
def  create_agent(model , action):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=action, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [31]:
print(env.observation_space.shape)  #  output (4,)


(4,)


In [32]:
dqn = create_agent(model, actions)

dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)

Training for 100000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 20:37 - reward: 1.0000

c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   38/10000 [..............................] - ETA: 4:12 - reward: 1.0000

c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is dep

10000/10000 [==============================] - 51s 5ms/step - reward: 1.0000
101 episodes - episode_reward: 98.861 [9.000, 396.000] - loss: 1.930 - mae: 19.144 - mean_q: 38.830

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 1.0000
35 episodes - episode_reward: 281.343 [184.000, 500.000] - loss: 3.218 - mae: 41.770 - mean_q: 84.589

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000
37 episodes - episode_reward: 267.405 [177.000, 430.000] - loss: 2.752 - mae: 47.314 - mean_q: 95.760

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 1.0000
40 episodes - episode_reward: 255.600 [189.000, 412.000] - loss: 1.826 - mae: 45.837 - mean_q: 92.636

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 56s 6ms/step - reward: 1.0000
35 episodes - episode_reward: 287.029 [201.000, 426.000] - loss: 

In [33]:
env = gym.make('CartPole-v1', render_mode='human') # if you want to render the environment and visualize it
results = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(results.history['episode_reward']))

env.close() 

Testing for 10 episodes ...


c:\Users\DELL\Desktop\ML\RL\venv\lib\site-packages\gym\core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500


KeyboardInterrupt: 